# **Data Loading**

In [1]:
!rm -r MURA-v1.1

rm: cannot remove 'MURA-v1.1': No such file or directory


In [2]:
#I uploaded our MURA dataset into my own dropbox. We run the line below to load the zip file of MURA dataset into google colab.
!wget https://www.dropbox.com/scl/fi/jkwwe1lstb6toum4oggis/MURA-v1.1.zip?rlkey=r9ab4hn1w5lsm2vijqw3a3wv6&st=rahx2fcg&dl=0
#Unzip the MURA dataset file
!unzip MURA-v1.1.zip?rlkey=r9ab4hn1w5lsm2vijqw3a3wv6&st=rahx2fcg&dl=0
#Remove zip file
!rm *zip*

Streaming output truncated to the last 5000 lines.
   creating: MURA-v1.1/train/XR_SHOULDER/patient01061/
   creating: MURA-v1.1/train/XR_SHOULDER/patient01061/study1_positive/
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01061/study1_positive/image1.png  
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01061/study1_positive/image3.png  
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01061/study1_positive/image2.png  
   creating: MURA-v1.1/train/XR_SHOULDER/patient01016/
   creating: MURA-v1.1/train/XR_SHOULDER/patient01016/study1_positive/
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01016/study1_positive/image1.png  
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01016/study1_positive/image3.png  
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01016/study1_positive/image2.png  
   creating: MURA-v1.1/train/XR_SHOULDER/patient01182/
   creating: MURA-v1.1/train/XR_SHOULDER/patient01182/study1_positive/
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01182/study1_positive/ima

In [3]:
#import necessary packages
#Basic
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import csv
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import math, random

#For confusion matrix
from sklearn.metrics import confusion_matrix, roc_curve, accuracy_score

#For loading images and processing images
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.image import per_image_standardization

#For rebuilding DenseNet169
from tensorflow.keras.models import Sequential
from keras.src.models import Functional
from tensorflow.keras.layers import Dense, AlphaDropout, Dropout, Activation, Flatten, GlobalAveragePooling2D, Input, Conv2D, MaxPooling2D,BatchNormalization, Concatenate, AveragePooling2D, ZeroPadding2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from keras.src.utils import file_utils

#For using image net
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input

In [4]:
#We focus on body site SHOULDER, you can change to other Body site, remember in captital letters
body_site = "SHOULDER"
#body_site = "HUMERUS"

#Resize the image to 96*96
image_size = (96, 96)
#image_size = (128, 128)

#Load images as RGB, remember Image Net uses RGB!!!Not grayscale, custom CNN can use grayscale and RGB both.
color_mode = "rgb"
#color_mode = "grayscale"

The folder MURA-v1.1 contains two folders: folders "train" and "valid" contain all the images for the training data set and the validation data set, respectively. The folder MURA-v1.1 also contains four .csv files: "train_image_paths.csv" and "valid_image_paths.csv" each contains one column for the paths to the images in the training data set and the validation data set, respectively; "train_labeled_studies.csv" and "valid_labeled_studies.csv" each contains two columns, one for the paths to the studies and the other for the labels (1 for positive and 0 for negative).



In [5]:
def load_data(body_site, image_size, color_mode):
    '''
    Load images (features) and labels (1 for positive and 0 for negative)
    for the training and validation set for a specific body site [body_site]
    with images having size [image_size] and color mode [color_mode].

    Args:
        body_site (str): the body site to focus on.
            Requires: has to be one of "ELBOW", "FINGER", "FOREARM", "HAND",
            "HUMERUS", "SHOULDER", and "WRIST". (Capitalization matters.)
        image_size ((int, int)): the image size.
        color_mode (str): the color mode to load the image.
            Requires: has to be one of "rgb" and "grayscale".

    Returns:
        train_X (4d np array): the training images for [body_site] with images
            of [image_size] in [color_mode]. The four dimensions are: the path
            to the image, the first dimension of [image_size], the second
            dimension of [image_size], and the channels in the [color_mode].
        train_y (4d np array): the validation images for [body_site] with
            images of [image_size] in [color_mode]. The four dimensions are:
            the path to the image, the first dimension of [image_size], the
            second dimension of [image_size], and the channels in the
            [color_mode].
        val_X (pd dataframe): the training labels for [body_site]. The indices
            of the dataframe are the paths to the images, and the labels are 1
            (positive) or 0 (negative).
        val_y (pd dataframe): the validation labels for [body_site]. The
            indices of the dataframe are the paths to the images, and the
            labels are 1 (positive) or 0 (negative).
    '''

    # get the paths for the images for the training set and validation set
    train_paths = pd.read_csv('MURA-v1.1/train_image_paths.csv', header = None, names = ["path"])
    val_paths = pd.read_csv('MURA-v1.1/valid_image_paths.csv', header = None, names = ["path"])
    # create labels: 0 for normal (negative) and 1 for abnormal (positive)
    train_paths['label'] = 0
    val_paths['label'] = 0
    train_paths.loc[train_paths['path'].str.contains('positive'), 'label'] = 1
    val_paths.loc[val_paths['path'].str.contains('positive'), 'label'] = 1

    # get the subset of images and labels for a particular body site
    train = train_paths[train_paths['path'].str.contains(body_site)]
    val = val_paths[val_paths['path'].str.contains(body_site)]

    # load training images as a 4 dimentional np array (image_id * image_size[0] * image_size[1] * color_mode)
    train_images = []
    for path in train['path']:
        img = load_img(path, target_size = image_size, color_mode = color_mode)
        #img_arr = per_image_standardization(img_to_array(img))
        img_arr = img_to_array(img)
        img_arr = per_image_standardization(img_arr)
        train_images.append(img_arr)
    train_X = np.array(train_images)

    # load validation images as a 4 dimentional np array (image_id * image_size[0] * image_size[1] * color_mode)
    val_images = []
    for path in val['path']:
        img = load_img(path, target_size = image_size, color_mode = color_mode)
        #img_arr = per_image_standardization(img_to_array(img))
        img_arr = img_to_array(img)
        img_arr = per_image_standardization(img_arr)
        val_images.append(img_arr)
    val_X = np.array(val_images)

    # set paths as indices such that the only column is for the labels
    train_y = train.set_index('path')
    val_y = val.set_index('path')

    # calculate number of observations and percentage
    train_count_all = len(train_y)
    train_count_normal = sum(train_y['label'] == 0)
    train_count_abnormal = sum(train_y['label'] == 1)
    train_count_percentage = round(train_count_abnormal * 100 / train_count_all, 2)
    val_count_all = len (val_y)
    val_count_normal = sum(val_y['label'] == 0)
    val_count_abnormal = sum(val_y['label'] == 1)
    val_count_percentage = round(val_count_abnormal * 100 / val_count_all, 2)

    # print relevant information
    print(f'We are loading {body_site} {color_mode} images with {image_size[0]}*{image_size[1]} image size.')
    print(f'The training set has a total of {train_count_all} images, of which {train_count_normal} are normal and {train_count_abnormal} are abnormal images. The percentage of abnormality is {train_count_percentage}%.')
    print(f'The validation set has a total of {val_count_all} images, of which {val_count_normal} are normal and {val_count_abnormal} are abnormal images. The percentage of abnormality is {val_count_percentage}%.')
    print(f'The training np array X has a dimension of {train_X.shape}')
    print(f'The validation np array X has a dimension of {val_X.shape}')
    print(f'The training label y has a dimension of {train_y.shape}')
    print(f'The validation label y has a dimension of {val_y.shape}')
    return train_X, train_y, val_X, val_y

In [6]:
def load_val_data(body_site, image_size, color_mode):
    '''
    Specify body site, image size, color mode above

    '''
    #Read in path file for training set and validation set
    val_paths = pd.read_csv('MURA-v1.1/valid_image_paths.csv',header=None,names=["path"])
    #create labels to classify normal and abnormal, first intiate all to be 0
    val_paths['label']=0
    #if find "positive" in path, set the label from 0 to 1
    val_paths.loc[val_paths['path'].str.contains('positive'),'label']=1

    #subset for body site images path
    val_y = val_paths[val_paths['path'].str.contains(body_site)]

    val_images = []
    for path in val_y['path']:
        img = load_img(path,target_size=image_size,color_mode=color_mode)
        #img_arr = per_image_standardization(img_to_array(img))
        img_arr = img_to_array(img)
        img_arr = per_image_standardization(img_arr)
        val_images.append(img_arr)
    val_X = np.array(val_images)

    val_y = val_y.set_index('path')

    return val_X, val_y



In [7]:
# load data
train_X, train_y, val_X, val_y = load_data(body_site, image_size, color_mode)

We are loading SHOULDER rgb images with 96*96 image size.
The training set has a total of 8379 images, of which 4211 are normal and 4168 are abnormal images. The percentage of abnormality is 49.74%.
The validation set has a total of 563 images, of which 285 are normal and 278 are abnormal images. The percentage of abnormality is 49.38%.
The training np array X has a dimension of (8379, 96, 96, 3)
The validation np array X has a dimension of (563, 96, 96, 3)
The training label y has a dimension of (8379, 1)
The validation label y has a dimension of (563, 1)


# **Kappa Statistic**

In [8]:
def cohen_kappa(y_truth, y_prob):
    '''
    Calculate Cohen's kappa statistic for the validaion set based on the
    model-output probabilities for y = 1, [y_prob], and the true labels (gold
    standard) [y_truth].

    Args:
        y_truth (np array): the true labels (gold standard) for the validation
            images.
        y_prob (np array): the model-output probabilities for y = 1 for the
            validation images. That is: model.predict(val_X), if [val_X] is the
            input images of the validation set.

    Returns:
        Kappa (float): Cohen's kappa statistic.

    '''
    # if probabilities > 0.5, then categorize as 1
    y_pred = y_prob > 0.5
    # calculate confusion matrix
    matrix = confusion_matrix(y_truth, y_pred)
    # calculate kappa statistic
    Expected_Accuracy = (matrix[0].sum() * matrix[:, 0].sum() + matrix[1].sum() * matrix[:, 1].sum()) / matrix.sum() ** 2
    Observed_Accuracy = (matrix[0, 0] + matrix[1, 1]) / matrix.sum()
    Kappa = (Observed_Accuracy - Expected_Accuracy) / (1 - Expected_Accuracy)
    return Kappa

# **Grad-CAM**

In [ ]:
#https://keras.io/examples/vision/grad_cam/
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)

    return heatmap.numpy()

def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    # Load the original image
    img = keras.preprocessing.image.load_img(img_path,color_mode=color_mode,target_size=(512,512))
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    #display(Image(cam_path))

def save_gradcam_n(n, a, img_paths, imgs, model, lastconv, modelname):

    for i in range(n):
        heatmap = make_gradcam_heatmap(np.expand_dims(imgs[i],axis=0), model, lastconv)
        pred = int(np.round(model.predict(np.expand_dims(imgs[i],axis=0))).flatten()[0])
        save_and_display_gradcam(img_paths[i], heatmap, cam_path=modelname+"_"+str(i)+"_"+str(pred)+".jpg", alpha=a)

def find_target_layer(m):
    last_conv_layer_name = list(filter(lambda x: isinstance(x, keras.layers.Conv2D), m.layers))[-1].name
    return last_conv_layer_name


# **Masking**

In [ ]:
import math
import random
import numpy as np

def mask_images(imgs, mask_size, maskn):
    '''
    Applies random square masks to a batch of images.

    Args:
        imgs (4d np array): Input images in the shape (batch_size, height, width, channels).
        mask_size (int): Size of the square mask to be applied.
        maskn (int): Number of masks to apply to each image.

    Returns:
        imgs2 (4d np array): Array of images with masks applied.
    '''

    # If no masking is required, return the original images unchanged
    if mask_size == 0 or maskn == 0:
        return imgs
    else:
        imgs2 = []  # Initialize a list to store the masked images

        # Calculate the number of maskable regions along one dimension
        num_masks = math.floor(float(imgs[0].shape[0]) / mask_size)

        # Iterate over each image in the batch
        for img in imgs:
            # Randomly select `maskn` regions for masking
            xy = random.sample([(x, y) for x in range(num_masks) for y in range(num_masks)], k=maskn)
            img2 = img.copy()  # Create a copy of the image to avoid modifying the original

            # Apply masks to the selected regions
            for x, y in xy:
                img2[(x * mask_size): ((x + 1) * mask_size), (y * mask_size): ((y + 1) * mask_size), :] = 0.0

            imgs2.append(img2)  # Add the masked image to the result list

        return np.array(imgs2)  # Convert the list back to a 4D numpy array


# **Rebuilding DenseNet169**

In [9]:
def dense_layer(x, growth_rate, name):
    '''
    One dense connectivity layer for a dense block.

    Args:
        x (tensor): input.
            Requires: the features should be in the last dimension.
        growth_rate (int): the number of output channels for the block
            (i.e. growth rate in the original paper).
        name (str): name for the layer.

    Returns:
        x (tensor): output for the dense connectivity layer.
    '''
    x1 = BatchNormalization(axis = -1, epsilon = 1.001e-5,
                            name = name + "_0_bn")(x)
    x1 = Activation("relu", name = name + "_0_relu")(x1)
    x1 = Conv2D(4 * growth_rate, 1, use_bias = False,
                name = name + "_1_conv")(x1)
    x1 = BatchNormalization(axis = -1, epsilon = 1.001e-5,
                            name = name + "_1_bn")(x1)
    x1 = Activation("relu", name = name + "_1_relu")(x1)
    x1 = Conv2D(growth_rate, 3, padding = "same", use_bias = False,
                name = name + "_2_conv")(x1)
    x = Concatenate(axis = -1, name = name + "_concat")([x, x1])
    return x

In [10]:
def dense_block(x, n_layers, name):
    '''
    A dense block.

    Args:
        x (tensor): input.
            Requires: the features should be in the last dimension.
        n_layers (int): the number of dense connectivity layers.
        name (str): name for the block.

    Returns:
        x (tensor): output for the dense block.
    '''
    for i in range(n_layers):
        x = dense_layer(x, 32, name = name + "_layer" + str(i + 1))
    return x


In [11]:
def transition_block(x, reduction, name):
    '''
    A transition block.

    Args:
        x (tensor): input.
            Requires: the features should be in the last dimension.
        reduction (float): compression factor at transition layers.
            Requires: has to be fall within (0, 1].
        name (str): name for the block.

    Returns:
        x (tensor): output for the block.
    '''

    x = BatchNormalization(axis = -1, epsilon = 1.001e-5,
                           name = name + "_bn")(x)
    x = Activation("relu", name = name + "_relu")(x)
    x = Conv2D(int(x.shape[-1] * reduction), 1, use_bias = False,
               name = name + "_conv")(x)
    x = AveragePooling2D(2, strides = 2, name = name + "_pool")(x)
    return x


In [12]:
DENSENET_PATH = "https://storage.googleapis.com/tensorflow/keras-applications/densenet/"
DENSENET169_PATH = DENSENET_PATH + "densenet169_weights_tf_dim_ordering_tf_kernels_notop.h5"
DENSENET201_PATH = DENSENET_PATH + "densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [13]:
def DenseNet_Modified(input_shape, densenet_name, weights = "imagenet"):
    '''
    Instantiates the DenseNet architecture.

    This function returns a Keras image classification model,
    optionally loaded with weights pre-trained on ImageNet.

    For image classification use cases, see
    [this page for detailed examples](
      https://keras.io/api/applications/#usage-examples-for-image-classification-models).

    For transfer learning use cases, make sure to read the
    [guide to transfer learning & fine-tuning](
      https://keras.io/guides/transfer_learning/).

    Note: each Keras Application expects a specific kind of input preprocessing.
    For DenseNet, call `keras.applications.densenet.preprocess_input`
    on your inputs before passing them to the model.
    `densenet.preprocess_input` will scale pixels between 0 and 1 and then
    will normalize each channel with respect to the ImageNet
    dataset statistics.

    Args:
        input_shape: shape tuple(otherwise the input shape
            has to be `(224, 224, 3)`
            (with `'channels_last'` data format)
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 32.
            E.g. `(200, 200, 3)` would be one valid value.
        densenet_name: "DenseNet169" or "DenseNet201"
        weights: one of `None` (random initialization),
            `"imagenet"` (pre-training on ImageNet),
            or the path to the weights file to be loaded.

    Returns:
        A model instance.
    '''

    if densenet_name == "DenseNet169":
        blocks = [6, 12, 32, 32]
    elif densenet_name == "DenseNet201":
        blocks = [6, 12, 48, 32]

    img_input = Input(shape = input_shape)

    x = ZeroPadding2D(padding = ((3, 3), (3, 3)))(img_input)
    x = Conv2D(64, 7, strides = 2, use_bias = False, name="conv1_conv")(x)
    x = BatchNormalization(axis = -1, epsilon = 1.001e-5, name = "conv1_bn")(x)
    x = Activation("relu", name = "conv1_relu")(x)
    x = ZeroPadding2D(padding = ((1, 1), (1, 1)))(x)
    x = MaxPooling2D(3, strides = 2, name = "pool1")(x)

    x = dense_block(x, blocks[0], name = "dense1")
    x = transition_block(x, 0.5, name = "trans1")
    x = dense_block(x, blocks[1], name = "dense2")
    x = transition_block(x, 0.5, name = "trans2")
    x = dense_block(x, blocks[2], name = "dense3")
    x = transition_block(x, 0.5, name = "trans3")
    x = dense_block(x, blocks[3], name = "dense4")

    x = BatchNormalization(axis = -1, epsilon = 1.001e-5, name = "bn")(x)
    x = Activation("relu", name = "relu")(x)

    inputs = img_input

    # Create model.
    model = Model(inputs, x)

    # Load weights.
    if weights == "imagenet":
        if densenet_name == "DenseNet169":
            weights_path = file_utils.get_file(
                "densenet169_weights_tf_dim_ordering_tf_kernels_notop.h5",
                DENSENET169_PATH,
                cache_subdir = "models",
                file_hash = "b8c4d4c20dd625c148057b9ff1c1176b")
        elif densenet_name == "DenseNet201":
            weights_path = file_utils.get_file(
                    "densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5",
                    DENSENET201_PATH,
                    cache_subdir="models",
                    file_hash="c13680b51ded0fb44dff2d8f86ac8bb1",
                )
        model.load_weights(weights_path)
    elif weights is not None:
        model.load_weights(weights)

    return model

# **DenseNet169 using imagenet weights**

In [14]:
image_shape = (96,96,3)

base_model = DenseNet_Modified(image_shape, "DenseNet169", weights = 'imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(1024, activation = 'relu')(x)
x = Dropout(0.2)(x)
x = Dense(1024, activation = 'relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(1, activation = 'sigmoid')(x)

model = Model(inputs = base_model.input, outputs = predictions)

for layer in base_model.layers:
    layer.trainable = False

# print(model.summary())
opt = Adam(learning_rate = 0.0001)
model.compile(optimizer = opt, loss = 'binary_crossentropy',
              metrics = ['accuracy', AUC(), cohen_kappa])

# print(model.summary())

datagen = ImageDataGenerator(rotation_range = 30, horizontal_flip = True)

traingen = datagen.flow(train_X, train_y, batch_size = 32, shuffle = True)

results_pre = model.fit(traingen, validation_data = (val_X, val_y),
                        steps_per_epoch = traingen.n // traingen.batch_size,
                        validation_batch_size = 32,
                        epochs = 10, verbose = 2)

51877672/51877672 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


TypeError: Expected sequence or array-like, got <class 'tensorflow.python.framework.ops.SymbolicTensor'>

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

opt = Adam(learning_rate = 1e-5)
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['accuracy',AUC()])

results_post = model.fit(traingen, validation_data = (val_X, val_y),
                         steps_per_epoch = traingen.n // traingen.batch_size,
                         validation_batch_size = 32,
                         epochs = 10, verbose = 2)

Epoch 1/10
261/261 - 90s - 345ms/step - accuracy: 0.7415 - auc_2: 0.8168 - loss: 0.5234 - val_accuracy: 0.7016 - val_auc_2: 0.7671 - val_loss: 0.5693
Epoch 2/10


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


261/261 - 1s - 3ms/step - accuracy: 0.7500 - auc_2: 0.7794 - loss: 0.5835 - val_accuracy: 0.6998 - val_auc_2: 0.7671 - val_loss: 0.5694
Epoch 3/10
261/261 - 25s - 94ms/step - accuracy: 0.7446 - auc_2: 0.8179 - loss: 0.5217 - val_accuracy: 0.7140 - val_auc_2: 0.7737 - val_loss: 0.5628
Epoch 4/10
261/261 - 1s - 2ms/step - accuracy: 0.7188 - auc_2: 0.8254 - loss: 0.5054 - val_accuracy: 0.7140 - val_auc_2: 0.7736 - val_loss: 0.5629
Epoch 5/10
261/261 - 25s - 95ms/step - accuracy: 0.7404 - auc_2: 0.8169 - loss: 0.5219 - val_accuracy: 0.7034 - val_auc_2: 0.7731 - val_loss: 0.5634
Epoch 6/10
261/261 - 1s - 2ms/step - accuracy: 0.6875 - auc_2: 0.7652 - loss: 0.5748 - val_accuracy: 0.7034 - val_auc_2: 0.7730 - val_loss: 0.5633
Epoch 7/10
261/261 - 42s - 160ms/step - accuracy: 0.7484 - auc_2: 0.8215 - loss: 0.5179 - val_accuracy: 0.7105 - val_auc_2: 0.7758 - val_loss: 0.5595
Epoch 8/10
261/261 - 1s - 2ms/step - accuracy: 0.8125 - auc_2: 0.8340 - loss: 0.5052 - val_accuracy: 0.7105 - val_auc_2: 0

In [ ]:
# base_model.trainable = True

opt = Adam(learning_rate = 1e-6)
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['accuracy',AUC()])

results_post = model.fit(traingen, validation_data = (val_X, val_y),
                         steps_per_epoch = traingen.n // traingen.batch_size,
                         validation_batch_size = 32,
                         epochs = 10, verbose = 2)

Epoch 1/10
261/261 - 72s - 274ms/step - accuracy: 0.7534 - auc_5: 0.8319 - loss: 0.5031 - val_accuracy: 0.5341 - val_auc_5: 0.5419 - val_loss: 0.8393
Epoch 2/10


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


261/261 - 1s - 5ms/step - accuracy: 0.6250 - auc_5: 0.7604 - loss: 0.6171 - val_accuracy: 0.5341 - val_auc_5: 0.5420 - val_loss: 0.8394
Epoch 3/10
261/261 - 24s - 92ms/step - accuracy: 0.7572 - auc_5: 0.8318 - loss: 0.5042 - val_accuracy: 0.5372 - val_auc_5: 0.5415 - val_loss: 0.8395
Epoch 4/10
261/261 - 1s - 3ms/step - accuracy: 0.7500 - auc_5: 0.8020 - loss: 0.5423 - val_accuracy: 0.5372 - val_auc_5: 0.5413 - val_loss: 0.8400
Epoch 5/10
261/261 - 42s - 160ms/step - accuracy: 0.7558 - auc_5: 0.8284 - loss: 0.5088 - val_accuracy: 0.5341 - val_auc_5: 0.5428 - val_loss: 0.8367
Epoch 6/10
261/261 - 1s - 2ms/step - accuracy: 0.8438 - auc_5: 0.9524 - loss: 0.3484 - val_accuracy: 0.5326 - val_auc_5: 0.5427 - val_loss: 0.8368
Epoch 7/10
261/261 - 39s - 148ms/step - accuracy: 0.7582 - auc_5: 0.8336 - loss: 0.5018 - val_accuracy: 0.5326 - val_auc_5: 0.5417 - val_loss: 0.8424
Epoch 8/10
261/261 - 1s - 2ms/step - accuracy: 0.5938 - auc_5: 0.6766 - loss: 0.6184 - val_accuracy: 0.5311 - val_auc_5: 

In [ ]:
# base_model.trainable = True

opt = Adam(learning_rate = 1e-7)
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['accuracy',AUC()])

results_post = model.fit(traingen, validation_data = (val_X, val_y),
                         steps_per_epoch = traingen.n // traingen.batch_size,
                         validation_batch_size = 32,
                         epochs = 10, verbose = 2)

Epoch 1/10
261/261 - 85s - 325ms/step - accuracy: 0.7543 - auc_6: 0.8315 - loss: 0.5056 - val_accuracy: 0.5387 - val_auc_6: 0.5434 - val_loss: 0.8435
Epoch 2/10


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


261/261 - 1s - 5ms/step - accuracy: 0.5938 - auc_6: 0.7814 - loss: 0.5290 - val_accuracy: 0.5387 - val_auc_6: 0.5436 - val_loss: 0.8435
Epoch 3/10
261/261 - 26s - 100ms/step - accuracy: 0.7540 - auc_6: 0.8330 - loss: 0.5031 - val_accuracy: 0.5372 - val_auc_6: 0.5431 - val_loss: 0.8442
Epoch 4/10
261/261 - 1s - 2ms/step - accuracy: 0.7812 - auc_6: 0.8843 - loss: 0.4282 - val_accuracy: 0.5357 - val_auc_6: 0.5432 - val_loss: 0.8441
Epoch 5/10
261/261 - 48s - 183ms/step - accuracy: 0.7509 - auc_6: 0.8265 - loss: 0.5104 - val_accuracy: 0.5357 - val_auc_6: 0.5450 - val_loss: 0.8392
Epoch 6/10
261/261 - 1s - 5ms/step - accuracy: 0.6562 - auc_6: 0.7157 - loss: 0.6994 - val_accuracy: 0.5357 - val_auc_6: 0.5448 - val_loss: 0.8393
Epoch 7/10
261/261 - 30s - 115ms/step - accuracy: 0.7584 - auc_6: 0.8313 - loss: 0.5052 - val_accuracy: 0.5372 - val_auc_6: 0.5432 - val_loss: 0.8425
Epoch 8/10
261/261 - 1s - 2ms/step - accuracy: 0.6875 - auc_6: 0.8254 - loss: 0.5175 - val_accuracy: 0.5372 - val_auc_6:

In [ ]:
for layer in base_model.layers:
    layer.trainable = True

for i, layer in enumerate(base_model.layers):
    print(i, layer.name, layer.trainable)

0 input_layer_1 True
1 zero_padding2d_2 True
2 conv1_conv True
3 conv1_bn True
4 conv1_relu True
5 zero_padding2d_3 True
6 pool1 True
7 dense1_layer1_0_bn True
8 dense1_layer1_0_relu True
9 dense1_layer1_1_conv True
10 dense1_layer1_1_bn True
11 dense1_layer1_1_relu True
12 dense1_layer1_2_conv True
13 dense1_layer1_concat True
14 dense1_layer2_0_bn True
15 dense1_layer2_0_relu True
16 dense1_layer2_1_conv True
17 dense1_layer2_1_bn True
18 dense1_layer2_1_relu True
19 dense1_layer2_2_conv True
20 dense1_layer2_concat True
21 dense1_layer3_0_bn True
22 dense1_layer3_0_relu True
23 dense1_layer3_1_conv True
24 dense1_layer3_1_bn True
25 dense1_layer3_1_relu True
26 dense1_layer3_2_conv True
27 dense1_layer3_concat True
28 dense1_layer4_0_bn True
29 dense1_layer4_0_relu True
30 dense1_layer4_1_conv True
31 dense1_layer4_1_bn True
32 dense1_layer4_1_relu True
33 dense1_layer4_2_conv True
34 dense1_layer4_concat True
35 dense1_layer5_0_bn True
36 dense1_layer5_0_relu True
37 dense1_layer5_

In [ ]:
from tensorflow.keras.applications.densenet import DenseNet169, preprocess_input

image_shape = (96,96,3)

base_model = DenseNet169(weights='imagenet', include_top=False, input_shape=image_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

print(model.summary())
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy',AUC()])

print(model.summary())

datagen = ImageDataGenerator(rotation_range=30,horizontal_flip=True)

traingen = datagen.flow(train_X, train_y, batch_size=32,shuffle=True)

results_pre=model.fit(traingen,validation_data=(val_X, val_y),steps_per_epoch=traingen.n//traingen.batch_size,validation_batch_size=32,epochs=10,verbose=2)

base_model.trainable = True

for i, layer in enumerate(base_model.layers):
    print(i, layer.name, layer.trainable)

model.compile(optimizer=Adam(learning_rate=1e-7), loss='binary_crossentropy', metrics=['accuracy',AUC()])

results_post=model.fit(traingen,validation_data=(val_X, val_y),steps_per_epoch=traingen.n//traingen.batch_size,validation_batch_size=32,epochs=10,verbose=2)

results_pre_df = pd.DataFrame(results_pre.history)
results_post_df = pd.DataFrame(results_post.history)
results_df = results_pre_df.append(results_post_df,sort=False)

with open('DN169_globalavgpool_2dense1024_aug_metrics.csv', 'w') as f:
    results_df.to_csv(f)

fpr, tpr, threshold = roc_curve(val_y,model.predict(val_X))
fpr = np.array(fpr)
tpr = np.array(tpr)
fpr_tpr = np.stack((fpr,tpr),axis=1)
header=['fpr','tpr']
with open('DN169_globalavgpool_2dense1024_aug_AUC.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(fpr_tpr)

kappa_conf=[]

for n in range(17):
    kappa_conf_temp=[n]
    val_X_mask=mask_images(val_X,24,n)
    kappa_conf_temp.append(cohen_kappa(y_prob=model.predict(val_X_mask),y_truth=val_y))
    conf_temp=confusion_matrix(val_y,np.round(model.predict(val_X_mask)))
    kappa_conf_temp.extend(conf_temp[0])
    kappa_conf_temp.extend(conf_temp[1])
    kappa_conf.append(kappa_conf_temp)

header=['masking','kappa','tn','fp','fn','tp']
with open('DN169_globalavgpool_2dense1024_aug_kappa_conf.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(kappa_conf)

print(find_target_layer(model))

save_gradcam_n(20,0.5,train_y.index[train_y['label']==1].to_numpy(),train_X[np.where(train_y["label"] == 1)[0]],model,find_target_layer(model),"DN169_train_pos")
save_gradcam_n(20,0.5,train_y.index[train_y['label']==0].to_numpy(),train_X[np.where(train_y["label"] == 0)[0]],model,find_target_layer(model),"DN169_train_neg")
save_gradcam_n(20,0.5,val_y.index[val_y['label']==1].to_numpy(),val_X[np.where(val_y["label"] == 1)[0]],model,find_target_layer(model),"DN169_val_pos")
save_gradcam_n(20,0.5,val_y.index[val_y['label']==0].to_numpy(),val_X[np.where(val_y["label"] == 0)[0]],model,find_target_layer(model),"DN169_val_neg")

from sklearn.metrics import accuracy_score

kappa_conf=[]

for site in ["WRIST","SHOULDER","HUMERUS","HAND","FOREARM","FINGER","ELBOW"]:
    val_X, val_y = load_val_data(site,image_size,color_mode)
    kappa_conf_temp=[site]
    kappa_conf_temp.append(accuracy_score(val_y,np.round(model.predict(val_X))))
    kappa_conf_temp.append(cohen_kappa(y_prob=model.predict(val_X),y_truth=val_y))
    conf_temp=confusion_matrix(val_y,np.round(model.predict(val_X)))
    kappa_conf_temp.extend(conf_temp[0])
    kappa_conf_temp.extend(conf_temp[1])
    kappa_conf.append(kappa_conf_temp)
    fpr, tpr, threshold = roc_curve(val_y,model.predict(val_X))
    fpr = np.array(fpr)
    tpr = np.array(tpr)
    fpr_tpr = np.stack((fpr,tpr),axis=1)
    header=['fpr','tpr']
    with open('DN169_globalavgpool_2dense1024_aug_AUC_'+site+'.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(fpr_tpr)

header=['site','accuracy','kappa','tn','fp','fn','tp']
with open('DN169_globalavgpool_2dense1024_aug_metrics_sites.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(kappa_conf)


# **Combine DenseNet169 with Boosting**

In [ ]:
image_shape = (96,96,3)

In [ ]:
def train_boosted_densenet(train_X, train_y, val_X, val_y,
                           num_models, image_shape):
    """
    Train a set of DenseNet models using a boosting-like approach.
    """
    models_list = []
    n_samples = len(train_X)
    # initialize sample weights to be uniform
    weights = np.ones(n_samples) / n_samples

    for i in range(num_models):
        print(f"Training model {i + 1}/{num_models}...")

        # Create a DenseNet model
        base_model = DenseNet169_Modified(image_shape, weights = 'imagenet')
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = BatchNormalization()(x)
        x = Flatten()(x)
        x = Dense(1024, activation = 'relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(1024, activation = 'relu')(x)
        x = Dropout(0.2)(x)
        predictions = Dense(1, activation = 'sigmoid')(x)

        model = Model(inputs = base_model.input, outputs = predictions)

        for layer in base_model.layers:
            layer.trainable = False

        opt = Adam(learning_rate = 0.0001)
        model.compile(optimizer = opt, loss = 'binary_crossentropy',
                      metrics = ['accuracy',AUC()])

        results = model.fit(train_X, train_y, sample_weight = weights,
                            validation_data = (val_X, val_y),
                            validation_batch_size = 32,
                            epochs = 2, verbose = 2)

        # Evaluate the model and adjust weights
        pred_y = np.argmax(model.predict(train_X), axis = 1)
        misclassified = (pred_y != train_y.values.flatten())

        # Update weights: increase for misclassified samples
        error_rate = np.sum(weights[misclassified]) / np.sum(weights)
        alpha = np.log((1 - error_rate) / (error_rate + 1e-10))  # Avoid divide by zero

        weights[misclassified] *= np.exp(alpha)  # Boost weights of misclassified samples
        weights /= np.sum(weights)  # Normalize weights

        # Store the model and its weight (alpha)
        models_list.append((model, error_rate, alpha))

    return models_list

In [ ]:
train_y.values.flatten().shape

(8379,)

In [ ]:
train_y

,label
path,
MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image1.png,1
MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image2.png,1
MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image3.png,1
MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/image1.png,1
MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/image2.png,1
...,...
MURA-v1.1/train/XR_SHOULDER/patient02693/study1_negative/image1.png,0
MURA-v1.1/train/XR_SHOULDER/patient02693/study1_negative/image2.png,0
MURA-v1.1/train/XR_SHOULDER/patient02693/study1_negative/image3.png,0


In [ ]:
def boosted_predict(models_list, X, y):
    """
    Make predictions using a set of boosted models.
    """
    predictions = np.zeros((X.shape[0], 2))
    for model, error_rate, alpha in models_list:
        predictions += alpha * model.predict(X)

        pred_y = np.argmax(predictions, axis = 1)
        errors = np.sum(pred_y != y)

        # Update weights: increase for misclassified samples
        error_rate = errors / y.shape[0]
        print(error_rate)
    return np.argmax(predictions, axis = 1)

In [ ]:
print(train_y.shape)

(8379, 1)


In [ ]:
num_models = 5  # Number of DenseNet models in the boosting ensemble

# Train boosted models
boosted_models = train_boosted_densenet(train_X, train_y, val_X, val_y,
                                        num_models, image_shape)
for model, error, alpha in boosted_models:
    print(f"Model Error: {error}, Alpha: {alpha}")


Training model 1/5...
Epoch 1/2
262/262 - 57s - 217ms/step - accuracy: 0.6297 - auc_2: 0.6740 - loss: 7.7542e-05 - val_accuracy: 0.7158 - val_auc_2: 0.7790 - val_loss: 0.5694
Epoch 2/2
262/262 - 49s - 188ms/step - accuracy: 0.7061 - auc_2: 0.7732 - loss: 6.8262e-05 - val_accuracy: 0.7158 - val_auc_2: 0.7814 - val_loss: 0.5643
262/262 ━━━━━━━━━━━━━━━━━━━━ 23s 55ms/step
Training model 2/5...
Epoch 1/2
262/262 - 64s - 246ms/step - accuracy: 0.6280 - auc_3: 0.6782 - loss: 7.7022e-05 - val_accuracy: 0.6963 - val_auc_3: 0.7529 - val_loss: 0.5874
Epoch 2/2
262/262 - 8s - 31ms/step - accuracy: 0.7094 - auc_3: 0.7733 - loss: 6.8194e-05 - val_accuracy: 0.6980 - val_auc_3: 0.7649 - val_loss: 0.5709
262/262 ━━━━━━━━━━━━━━━━━━━━ 25s 59ms/step
Training model 3/5...
Epoch 1/2
262/262 - 66s - 250ms/step - accuracy: 0.6249 - auc_4: 0.6762 - loss: 7.6957e-05 - val_accuracy: 0.6803 - val_auc_4: 0.7486 - val_loss: 0.5924
Epoch 2/2
262/262 - 38s - 146ms/step - accuracy: 0.7109 - auc_4: 0.7772 - loss: 6.767

In [ ]:
pred_val_y = boosted_predict(boosted_models, val_X, val_y.values.flatten())

# print("Predictions:", pred_val_y)

18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step
0.4937833037300178
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
0.4937833037300178
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
0.4937833037300178
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
0.4937833037300178
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
0.4937833037300178


# **Saving output for plotting**

In [ ]:
results_df = pd.DataFrame(results.history)

with open('4convmaxpool_batchnorm_4dense_aug_metrics.csv', 'w') as f:
    results_df.to_csv(f)

fpr, tpr, threshold = roc_curve(val_y,model.predict(val_X))
fpr = np.array(fpr)
tpr = np.array(tpr)
fpr_tpr = np.stack((fpr,tpr),axis=1)
header=['fpr','tpr']
with open('4convmaxpool_batchnorm_4dense_aug_AUC.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(fpr_tpr)


kappa_conf=[]

for n in range(17):
    kappa_conf_temp=[n]
    val_X_mask=mask_images(val_X,24,n)
    kappa_conf_temp.append(cohen_kappa(y_prob=model.predict(val_X_mask),y_truth=val_y))
    conf_temp=confusion_matrix(val_y,np.round(model.predict(val_X_mask)))
    kappa_conf_temp.extend(conf_temp[0])
    kappa_conf_temp.extend(conf_temp[1])
    kappa_conf.append(kappa_conf_temp)

header=['masking','kappa','tn','fp','fn','tp']
with open('4convmaxpool_batchnorm_4dense_aug_kappa_conf.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(kappa_conf)

# **Saving Grad-CAM images**

In [ ]:

save_gradcam_n(20,0.5,train_y.index[train_y['label']==1].to_numpy(),train_X[np.where(train_y["label"] == 1)[0]],model,find_target_layer(model),"custom_train_pos")
save_gradcam_n(20,0.5,train_y.index[train_y['label']==0].to_numpy(),train_X[np.where(train_y["label"] == 0)[0]],model,find_target_layer(model),"custom_train_neg")
save_gradcam_n(20,0.5,val_y.index[val_y['label']==1].to_numpy(),val_X[np.where(val_y["label"] == 1)[0]],model,find_target_layer(model),"custom_val_pos")
save_gradcam_n(20,0.5,val_y.index[val_y['label']==0].to_numpy(),val_X[np.where(val_y["label"] == 0)[0]],model,find_target_layer(model),"custom_val_neg")


In [ ]:
#calculate cohen_kappa
cohen_kappa(y_prob=model.predict(val_X),y_truth=val_y)

0.09523749503708778

# **Transfer to other body sites**

In [ ]:
kappa_conf=[]

for site in ["WRIST","SHOULDER","HUMERUS","HAND","FOREARM","FINGER","ELBOW"]:
    val_X, val_y = load_val_data(site,image_size,color_mode)
    kappa_conf_temp=[site]
    kappa_conf_temp.append(accuracy_score(val_y,np.round(model.predict(val_X))))
    kappa_conf_temp.append(cohen_kappa(y_prob=model.predict(val_X),y_truth=val_y))
    conf_temp=confusion_matrix(val_y,np.round(model.predict(val_X)))
    kappa_conf_temp.extend(conf_temp[0])
    kappa_conf_temp.extend(conf_temp[1])
    kappa_conf.append(kappa_conf_temp)
    fpr, tpr, threshold = roc_curve(val_y,model.predict(val_X))
    fpr = np.array(fpr)
    tpr = np.array(tpr)
    fpr_tpr = np.stack((fpr,tpr),axis=1)
    header=['fpr','tpr']
    with open('4convmaxpool_batchnorm_4dense_aug_AUC_'+site+'.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(fpr_tpr)

header=['site','accuracy','kappa','tn','fp','fn','tp']
with open('4convmaxpool_batchnorm_4dense_aug_metrics_sites.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(kappa_conf)


NameError: name 'model' is not defined

# **VGG16 using imagenet weights**

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

image_shape = (96,96,3)

base_model = VGG16(weights='imagenet', include_top=False, input_shape=image_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

print(model.summary())
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy',AUC()])

print(model.summary())

datagen = ImageDataGenerator(rotation_range=30,horizontal_flip=True)

traingen = datagen.flow(train_X, train_y, batch_size=32,shuffle=True)

results_pre=model.fit(traingen,validation_data=(val_X, val_y),steps_per_epoch=traingen.n//traingen.batch_size,validation_batch_size=32,epochs=10,verbose=2)

base_model.trainable = True


for i, layer in enumerate(base_model.layers):
    print(i, layer.name, layer.trainable)

model.compile(optimizer=Adam(learning_rate=1e-7), loss='binary_crossentropy', metrics=['accuracy',AUC()])

results_post=model.fit(traingen,validation_data=(val_X, val_y),steps_per_epoch=traingen.n//traingen.batch_size,validation_batch_size=32,epochs=10,verbose=2)

results_pre_df = pd.DataFrame(results_pre.history)
results_post_df = pd.DataFrame(results_post.history)
results_df = results_pre_df.append(results_post_df,sort=False)

with open('VGG16_globalavgpool_2dense1024_aug_metrics.csv', 'w') as f:
    results_df.to_csv(f)

fpr, tpr, threshold = roc_curve(val_y,model.predict(val_X))
fpr = np.array(fpr)
tpr = np.array(tpr)
fpr_tpr = np.stack((fpr,tpr),axis=1)
header=['fpr','tpr']
with open('VGG16_globalavgpool_2dense1024_aug_AUC.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(fpr_tpr)

kappa_conf=[]

for n in range(17):
    kappa_conf_temp=[n]
    val_X_mask=mask_images(val_X,24,n)
    kappa_conf_temp.append(cohen_kappa(y_prob=model.predict(val_X_mask),y_truth=val_y))
    conf_temp=confusion_matrix(val_y,np.round(model.predict(val_X_mask)))
    kappa_conf_temp.extend(conf_temp[0])
    kappa_conf_temp.extend(conf_temp[1])
    kappa_conf.append(kappa_conf_temp)

header=['masking','kappa','tn','fp','fn','tp']
with open('VGG16_globalavgpool_2dense1024_aug_kappa_conf.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(kappa_conf)

print(find_target_layer(model))

save_gradcam_n(20,0.5,train_y.index[train_y['label']==1].to_numpy(),train_X[np.where(train_y["label"] == 1)[0]],model,find_target_layer(model),"VGG16_train_pos")
save_gradcam_n(20,0.5,train_y.index[train_y['label']==0].to_numpy(),train_X[np.where(train_y["label"] == 0)[0]],model,find_target_layer(model),"VGG16_train_neg")
save_gradcam_n(20,0.5,val_y.index[val_y['label']==1].to_numpy(),val_X[np.where(val_y["label"] == 1)[0]],model,find_target_layer(model),"VGG16_val_pos")
save_gradcam_n(20,0.5,val_y.index[val_y['label']==0].to_numpy(),val_X[np.where(val_y["label"] == 0)[0]],model,find_target_layer(model),"VGG16_val_neg")

from sklearn.metrics import accuracy_score

kappa_conf=[]

for site in ["WRIST","SHOULDER","HUMERUS","HAND","FOREARM","FINGER","ELBOW"]:
    val_X, val_y = load_val_data(site,image_size,color_mode)
    kappa_conf_temp=[site]
    kappa_conf_temp.append(accuracy_score(val_y,np.round(model.predict(val_X))))
    kappa_conf_temp.append(cohen_kappa(y_prob=model.predict(val_X),y_truth=val_y))
    conf_temp=confusion_matrix(val_y,np.round(model.predict(val_X)))
    kappa_conf_temp.extend(conf_temp[0])
    kappa_conf_temp.extend(conf_temp[1])
    kappa_conf.append(kappa_conf_temp)
    fpr, tpr, threshold = roc_curve(val_y,model.predict(val_X))
    fpr = np.array(fpr)
    tpr = np.array(tpr)
    fpr_tpr = np.stack((fpr,tpr),axis=1)
    header=['fpr','tpr']
    with open('VGG16_globalavgpool_2dense1024_aug_AUC_'+site+'.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(fpr_tpr)

header=['site','accuracy','kappa','tn','fp','fn','tp']
with open('VGG16_globalavgpool_2dense1024_aug_metrics_sites.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(kappa_conf)

Epoch 1/10
262/262 [==============================] - 13s 34ms/step - loss: 5.6578 - accuracy: 0.5174 - auc_3: 0.5200 - val_loss: 6.4474 - val_accuracy: 0.4956 - val_auc_3: 0.5096
Epoch 2/10
262/262 [==============================] - 7s 28ms/step - loss: 5.0836 - accuracy: 0.5279 - auc_3: 0.5365 - val_loss: 2.8441 - val_accuracy: 0.5950 - val_auc_3: 0.6246
Epoch 3/10
262/262 [==============================] - 7s 28ms/step - loss: 4.9112 - accuracy: 0.5374 - auc_3: 0.5446 - val_loss: 4.0973 - val_accuracy: 0.5044 - val_auc_3: 0.6005
Epoch 4/10
262/262 [==============================] - 7s 27ms/step - loss: 4.7065 - accuracy: 0.5403 - auc_3: 0.5515 - val_loss: 2.2037 - val_accuracy: 0.6128 - val_auc_3: 0.6267
Epoch 5/10
262/262 [==============================] - 7s 28ms/step - loss: 4.6530 - accuracy: 0.5466 - auc_3: 0.5641 - val_loss: 2.4845 - val_accuracy: 0.5435 - val_auc_3: 0.6225
Epoch 6/10
262/262 [==============================] - 7s 29ms/step - loss: 4.6692 - accuracy: 0.5390 - a